# CLAU Banking API Deployment Checklist

Use this checklist to ensure you've completed all the necessary steps for deploying the CLAU Banking Intelligence API stack to AWS EC2.

## Pre-Deployment

- [ ] Repository prepared and organized
  - [ ] Frontend and backend code separated
  - [ ] All dependencies listed in package.json files
  - [ ] Environment variable templates prepared (.env.example files)
  - [ ] README files updated with deployment instructions

- [ ] AWS account setup
  - [ ] AWS account created and verified
  - [ ] Budget alerts set up (optional but recommended)
  - [ ] IAM user with appropriate permissions created

- [ ] Domain and certificates (optional)
  - [ ] Domain registered (Route 53 or third-party registrar)
  - [ ] SSL certificate obtained or prepared to use Let's Encrypt

## AWS Infrastructure Setup

- [ ] EC2 instance launched
  - [ ] Appropriate instance type selected (t2.micro, t2.small, etc.)
  - [ ] Amazon Linux 2023 AMI selected
  - [ ] Key pair created and downloaded
  - [ ] Security group created with proper rules
  - [ ] Elastic IP allocated and associated (optional)

- [ ] Security group configured
  - [ ] SSH (Port 22) access limited to your IP
  - [ ] HTTP (Port 80) opened
  - [ ] HTTPS (Port 443) opened
  - [ ] Custom TCP ports opened if needed (3000, 3001)

## Server Configuration

- [ ] Connected to EC2 instance via SSH
- [ ] System updated
  ```bash
  sudo yum update -y
  ```
- [ ] Required software installed
  - [ ] Node.js and npm (via nvm)
  - [ ] Git
  - [ ] PM2 (process manager)
  - [ ] Nginx
  - [ ] Certbot (if using Let's Encrypt)

## Backend Deployment

- [ ] Backend code transferred to EC2
  - [ ] Directory created: `~/clau-api/backend`
  - [ ] Code cloned or uploaded
- [ ] Dependencies installed
  ```bash
  npm install
  ```
- [ ] Environment variables configured
  - [ ] .env file created with necessary variables
  - [ ] JWT secrets securely generated
  - [ ] Cohere API key set
  - [ ] Other required API keys set
- [ ] Database setup
  - [ ] SQLite file created (for development)
  - [ ] Or PostgreSQL/MySQL configured (for production)
- [ ] Application started with PM2
  ```bash
  pm2 start server.js --name clau-api-backend
  ```
- [ ] PM2 startup configured
  ```bash
  pm2 startup
  pm2 save
  ```
- [ ] API tested
  ```bash
  curl http://localhost:3001/api/health
  ```

## Frontend Deployment

- [ ] Frontend code transferred to EC2
  - [ ] Directory created: `~/clau-api/frontend`
  - [ ] Code cloned or uploaded
- [ ] Dependencies installed
  ```bash
  npm install
  ```
- [ ] API URL configured in .env file
  ```
  REACT_APP_API_URL=http://your-instance-public-dns:3001/api
  ```
- [ ] Production build created
  ```bash
  npm run build
  ```
- [ ] Serve installed and started with PM2
  ```bash
  npm install -g serve
  pm2 start serve --name clau-api-frontend -- -s build -l 3000
  pm2 save
  ```
- [ ] Frontend tested
  ```bash
  curl http://localhost:3000
  ```

## Nginx Configuration

- [ ] Nginx configuration file created
  ```bash
  sudo nano /etc/nginx/conf.d/clau-api.conf
  ```
- [ ] Configuration tested
  ```bash
  sudo nginx -t
  ```
- [ ] Nginx started and enabled
  ```bash
  sudo systemctl start nginx
  sudo systemctl enable nginx
  ```

## HTTPS Setup (Optional but Recommended)

- [ ] Certbot installed
- [ ] SSL certificate obtained
  ```bash
  sudo certbot --nginx -d your-domain.com
  ```
- [ ] Auto-renewal configured
  ```bash
  sudo systemctl enable certbot-renew.timer
  ```

## Final Verification

- [ ] Nginx is running and accessible
  ```bash
  sudo systemctl status nginx
  curl http://your-instance-public-dns
  ```
- [ ] Backend API is accessible
  ```bash
  curl http://your-instance-public-dns/api/health
  ```
- [ ] Frontend is accessible in a browser
  - [ ] Navigate to http://your-instance-public-dns
  - [ ] Test basic functionality
- [ ] Login and registration work
- [ ] API requests are successful

## Post-Deployment Setup

- [ ] Monitoring configured
  - [ ] PM2 monitoring enabled
  - [ ] CloudWatch alarms set up (optional)
  
- [ ] Backups configured
  - [ ] Backup script created
  - [ ] Cron job scheduled

- [ ] Deployment automation (Optional)
  - [ ] GitHub Actions workflows configured
  - [ ] Deployment script created

## Documentation

- [ ] Deployment process documented
- [ ] Environment variables documented
- [ ] Endpoints documented
- [ ] Troubleshooting guide created

## Security Hardening (Production)

- [ ] Firewall configured
  ```bash
  sudo yum install -y firewalld
  sudo systemctl start firewalld
  sudo systemctl enable firewalld
  sudo firewall-cmd --permanent --add-service=http
  sudo firewall-cmd --permanent --add-service=https
  sudo firewall-cmd --permanent --add-service=ssh
  sudo firewall-cmd --reload
  ```
- [ ] Regular updates scheduled
  ```bash
  sudo yum install -y yum-cron
  sudo systemctl enable yum-cron
  sudo systemctl start yum-cron
  ```
- [ ] Fail2ban installed (optional, for SSH protection)
  ```bash
  sudo yum install epel-release -y
  sudo yum install fail2ban -y
  sudo systemctl enable fail2ban
  sudo systemctl start fail2ban
  ```

## Scaling and Improvements (Future)

- [ ] Load balancer setup planned
- [ ] Auto-scaling configured (if needed)
- [ ] Database migration to RDS planned
- [ ] CDN integration planned
- [ ] Monitoring and alerting enhanced

## Common Issues Checklist

If you encounter issues during deployment, check these common problem areas:

- [ ] Correct Node.js version installed (should be compatible with your application)
- [ ] All dependencies installed (no missing peer dependencies)
- [ ] Environment variables correctly set (check for typos)
- [ ] File permissions are correct (especially for SQLite database)
- [ ] Ports are open in the security group
- [ ] Nginx configuration is correct
- [ ] SSL certificates are valid and not expired

## Rollback Plan

In case of deployment failure, have a rollback plan:

- [ ] Previous version backed up
- [ ] Rollback commands documented
- [ ] Database rollback procedure documented

# Deploying CLAU Banking Intelligence API on AWS EC2

## Step 1: Launch an EC2 Instance

1. **Sign in to AWS Console**
   - Go to [AWS Console](https://console.aws.amazon.com/)
   - Navigate to EC2 service

2. **Launch a new EC2 instance**
   - Click "Launch Instance"
   - Name your instance (e.g., "clau-banking-api-server")
   
3. **Choose AMI and Instance Type**
   - Select Amazon Linux 2023 AMI (free tier eligible)
   - Choose t2.micro for free tier or t2.small/t2.medium for better performance
   
4. **Configure Security Group**
   - Create a new security group named "CLAU-API-SG"
   - Add the following inbound rules:
     - SSH (Port 22): Your IP
     - HTTP (Port 80): Anywhere (0.0.0.0/0)
     - HTTPS (Port 443): Anywhere (0.0.0.0/0)
     - Custom TCP (Port 3000): Anywhere (for React frontend)
     - Custom TCP (Port 3001): Anywhere (for Node.js backend)

5. **Add Storage**
   - Default 8GB is sufficient to start
   
6. **Launch Instance**
   - Create a new key pair named "clau-api-key"
   - Download the key pair (.pem file)
   - Launch the instance

## Step 2: Connect to your EC2 Instance

1. **Open Terminal/Command Prompt on your local machine**

2. **Change permissions for your key file (macOS/Linux only)**
   ```bash
   chmod 400 path/to/clau-api-key.pem
   ```

3. **Connect to your instance**
   ```bash
   ssh -i path/to/clau-api-key.pem ec2-user@your-instance-public-dns
   ```
   Replace `your-instance-public-dns` with your actual EC2 public DNS

## Step 3: Install Required Software

1. **Update the system**
   ```bash
   sudo yum update -y
   ```

2. **Install Node.js and npm**
   ```bash
   curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.5/install.sh | bash
   source ~/.bashrc
   nvm install 16  # Install Node.js v16 (LTS) - compatible with your application
   ```

3. **Install Git**
   ```bash
   sudo yum install git -y
   ```

4. **Install PM2 (process manager) globally**
   ```bash
   npm install -g pm2
   ```

5. **Install Nginx (for reverse proxy)**
   ```bash
   sudo amazon-linux-extras install nginx1 -y
   # If above command doesn't work on Amazon Linux 2023, use:
   sudo yum install nginx -y
   ```

## Step 4: Set Up the Backend Application

1. **Create a directory for the backend**
   ```bash
   mkdir -p ~/clau-api/backend
   cd ~/clau-api/backend
   ```

2. **Clone your repository or upload your files**
   ```bash
   # Option 1: Clone from Git repository (if available)
   git clone your-backend-repo-url .
   
   # Option 2: Upload files using SCP (run this from your local machine)
   # scp -i path/to/clau-api-key.pem -r /path/to/local/backend/* ec2-user@your-instance-public-dns:~/clau-api/backend/
   ```

3. **Install dependencies**
   ```bash
   npm install
   ```

4. **Create .env file**
   ```bash
   nano .env
   ```
   
   Add the necessary environment variables:
   ```
   NODE_ENV=production
   PORT=3001
   JWT_SECRET=your-secure-secret-key
   JWT_REFRESH_SECRET=your-secure-refresh-key
   COHERE_API_KEY=your-cohere-api-key
   # Add all other necessary environment variables here
   ```
   
   Save and exit (Ctrl+X, then Y, then Enter)

5. **Start the backend with PM2**
   ```bash
   pm2 start server.js --name clau-api-backend
   ```

6. **Configure PM2 to start on system boot**
   ```bash
   pm2 startup
   # Run the command it outputs
   pm2 save
   ```

## Step 5: Set Up the Frontend Application

1. **Create a directory for the frontend**
   ```bash
   mkdir -p ~/clau-api/frontend
   cd ~/clau-api/frontend
   ```

2. **Clone your repository or upload your files**
   ```bash
   # Option 1: Clone from Git repository (if available)
   git clone your-frontend-repo-url .
   
   # Option 2: Upload files using SCP (run this from your local machine)
   # scp -i path/to/clau-api-key.pem -r /path/to/local/frontend/* ec2-user@your-instance-public-dns:~/clau-api/frontend/
   ```

3. **Install dependencies**
   ```bash
   npm install
   ```

4. **Update API endpoint in the frontend**
   - Locate the API configuration file (likely `/src/services/api.js`)
   ```bash
   nano src/services/api.js
   ```
   - Update the API URL to point to your backend:
   ```javascript
   const API_URL = process.env.REACT_APP_API_URL || 'http://your-instance-public-dns:3001/api';
   ```
   - Save and exit (Ctrl+X, then Y, then Enter)

5. **Create .env file for the frontend**
   ```bash
   nano .env
   ```
   
   Add:
   ```
   REACT_APP_API_URL=http://your-instance-public-dns:3001/api
   ```
   
   Save and exit (Ctrl+X, then Y, then Enter)

6. **Build the React app**
   ```bash
   npm run build
   ```

7. **Serve the built files using PM2 and serve**
   ```bash
   npm install -g serve@13.0.4
   pm2 start "serve -s build -l 3000" --name clau-api-frontend
   pm2 save
   ```

## Step 6: Set Up Nginx as a Reverse Proxy

1. **Create Nginx configuration file**
   ```bash
   sudo nano /etc/nginx/conf.d/clau-api.conf
   ```

2. **Add the following configuration**
   ```nginx
   server {
       listen 80;
       server_name your-instance-public-dns;  # Replace with your domain if available

       # Frontend
       location / {
           proxy_pass http://localhost:3001;
           proxy_http_version 1.1;
           proxy_set_header Upgrade $http_upgrade;
           proxy_set_header Connection 'upgrade';
           proxy_set_header Host $host;
           proxy_cache_bypass $http_upgrade;
       }

       # Backend API
       location /api/ {
           proxy_pass http://localhost:3000/api/;
           proxy_http_version 1.1;
           proxy_set_header Upgrade $http_upgrade;
           proxy_set_header Connection 'upgrade';
           proxy_set_header Host $host;
           proxy_cache_bypass $http_upgrade;
       }
   }
   ```
   
   Save and exit (Ctrl+X, then Y, then Enter)

3. **Test the Nginx configuration**
   ```bash
   sudo nginx -t
   ```

4. **Start or restart Nginx**
   ```bash
   sudo systemctl start nginx
   sudo systemctl enable nginx
   ```

## Step 7: Set Up HTTPS with Let's Encrypt (Optional but Recommended)

1. **Install Certbot**
   ```bash
   sudo amazon-linux-extras install epel -y
   # If the above command doesn't work on Amazon Linux 2023, use:
   sudo dnf install epel-release -y

   sudo yum install certbot python-certbot-nginx -y
   ```

2. **Obtain SSL certificate**
   - For a domain pointing to your EC2 instance:
   ```bash
   sudo certbot --nginx -d your-domain.com
   ```
   
   - If you don't have a domain and want to use your EC2 public DNS (not recommended for production):
   ```bash
   sudo yum install certbot -y
   sudo certbot certonly --standalone -d your-instance-public-dns
   ```

3. **Set up auto-renewal**
   ```bash
   sudo systemctl enable certbot-renew.timer
   ```

## Step 8: Set Up a SQLite Database for Development (Optional)

Since your backend is designed to work with SQLite in development mode:

```bash
cd ~/clau-api/backend
touch database.sqlite
chmod 666 database.sqlite
```

## Step 9: Verify Deployment

1. **Check PM2 processes**
   ```bash
   pm2 status
   ```

2. **Check Nginx status**
   ```bash
   sudo systemctl status nginx
   ```

3. **Test the frontend**
   - Open a web browser and navigate to:
   ```
   http://your-instance-public-dns
   ```

4. **Test the backend API**
   - Open a web browser and navigate to:
   ```
   http://your-instance-public-dns/api/health
   ```

## Step 10: Troubleshooting

1. **Check backend logs**
   ```bash
   pm2 logs clau-api-backend
   ```

2. **Check frontend logs**
   ```bash
   pm2 logs clau-api-frontend
   ```

3. **Check Nginx logs**
   ```bash
   sudo cat /var/log/nginx/error.log
   sudo cat /var/log/nginx/access.log
   ```

4. **Check Nginx configuration**
   ```bash
   sudo nginx -t
   ```

## Step 11: Setting Up a Custom Domain (Optional)

1. **Register a domain** (if you don't have one)
   - Use a domain registrar like Route 53, GoDaddy, or Namecheap

2. **Create an A record**
   - Point your domain to your EC2 instance's public IP address

3. **Update Nginx configuration**
   ```bash
   sudo nano /etc/nginx/conf.d/clau-api.conf
   ```
   
   Update `server_name` to your domain name
   
4. **Restart Nginx**
   ```bash
   sudo systemctl restart nginx
   ```

5. **Set up HTTPS with your domain**
   ```bash
   sudo certbot --nginx -d your-domain.com
   ```

## Step 12: Set Up Automated Deployments (Optional)

For simpler future deployments, you can set up a deployment script:

1. **Create a deployment script**
   ```bash
   nano ~/deploy.sh
   ```

2. **Add deployment commands**
   ```bash
   #!/bin/bash

   cd ~/clau-api/MAIN-WITH-UI/
   git pull
   
   # Update backend
   cd ~/clau-api/MAIN-WITH-UI/"API Code (Backend)"
   npm install
   pm2 restart clau-api-backend

   # Update frontend
   cd ~/clau-api/MAIN-WITH-UI/"API UI (Frontend)"
   npm install
   npm install -g server@13.0.4
   npm run build
   pm2 restart clau-api-frontend
   ```

3. **Make the script executable**
   ```bash
   chmod +x ~/deploy.sh
   ```

4. **Run the script to deploy updates**
   ```bash
   ~/deploy.sh
   ```

## Step 13: Back Up Your Data

1. **Set up a regular backup of your SQLite database**
   ```bash
   mkdir -p ~/backups
   ```

2. **Create a backup script**
   ```bash
   nano ~/backup.sh
   ```

3. **Add backup commands**
   ```bash
   #!/bin/bash
   DATE=$(date +%Y-%m-%d-%H-%M)
   cp ~/clau-api/backend/database.sqlite ~/backups/database-$DATE.sqlite
   ```

4. **Make the script executable**
   ```bash
   chmod +x ~/backup.sh
   ```

5. **Schedule regular backups with cron**
   ```bash
   crontab -e
   ```
   
   Add:
   ```
   0 0 * * * ~/backup.sh
   ```
   
   This will run the backup daily at midnight.

## Conclusion

You have successfully deployed both the frontend and backend of the CLAU Banking Intelligence API on an AWS EC2 instance. To deploy changes in the future, you can use the deployment script or manually update each part of the application.

For a production environment, you should consider:
- Setting up a proper database like PostgreSQL or MySQL
- Implementing more robust security measures
- Using a managed environment like Elastic Beanstalk
- Setting up monitoring and logging with CloudWatch

Remember to regularly backup your data and keep your server updated with security patches.

flowchart TD
    subgraph "AWS Cloud"
        EC2["EC2 Instance\nt2.small or t2.medium"]
        EIP["Elastic IP"]
        SG["Security Group"]
        
        subgraph "EC2 Instance"
            PM2["PM2 Process Manager"]
            NGINX["Nginx Web Server/Proxy"]
            
            subgraph "Frontend"
                REACT["React App\n(Port 3000)"]
            end
            
            subgraph "Backend"
                NODE["Node.js API Server\n(Port 3001)"]
                SQLITE["SQLite Database\n(Development)"]
                JWT["JWT Authentication"]
                COHERE["Cohere AI Service"]
            end
        end
        
        EIP --> EC2
        SG --> EC2
        
        PM2 --> REACT
        PM2 --> NODE
        NGINX --> REACT
        NGINX --> NODE
        NODE --> SQLITE
        NODE --> JWT
        NODE --> COHERE
    end
    
    INTERNET["Internet"] --> EIP
    
    BROWSER["User Browser"] --> INTERNET
    
    subgraph "External Services"
        COHEREAPI["Cohere API"]
    end
    
    COHERE --> COHEREAPI
    
    subgraph "Future Enhancements"
        style Future fill:#f9f,stroke:#333,stroke-dasharray: 5 5
        RDS["Amazon RDS\n(PostgreSQL)"]
        S3["Amazon S3\n(Asset Storage)"]
        CLOUDFRONT["CloudFront\n(CDN)"]
        ROUTE53["Route 53\n(DNS)"]
        ACM["Certificate Manager\n(SSL/TLS)"]
        
        NODE -.-> RDS
        REACT -.-> S3
        CLOUDFRONT -.-> S3
        INTERNET -.-> CLOUDFRONT
        ROUTE53 -.-> EIP
        ROUTE53 -.-> CLOUDFRONT
        ACM -.-> NGINX
    end
    
    classDef current fill:#f5f5f5,stroke:#333,stroke-width:2px;
    classDef external fill:#bbf,stroke:#333,stroke-width:1px;
    classDef future fill:#ffe,stroke:#333,stroke-width:1px,stroke-dasharray: 5 5;
    
    class EC2,EIP,SG,PM2,NGINX,REACT,NODE,SQLITE,JWT,COHERE current;
    class BROWSER,INTERNET,COHEREAPI external;
    class RDS,S3,CLOUDFRONT,ROUTE53,ACM future;